# Limpeza dos Arquivos

Os dados disponibilizados no site da Câmara dos Deputados são dividos em diversos arquivos e não estão previamente organizados para nossos objetivos. Sendo assim, é necessário realizar uma limpeza nos arquivos que serão utilizados, pois iremos juntar dados de diferentes arquivos que possuem algumas categorias irrelevantes. 

Esse arquivo jupyter é responsável por documentar e realizar a limpeza dos dados baixados, além de gerar as bases de dados que realmente serão utilizadas para criação de redes e análises.

* Observação : Estamos na 56º Legislatura (2019 - 2023).

Importação das bibliotecas necessárias pra limpeza.

In [28]:
import pandas as pd
import requests
import os
from unidecode import unidecode

Listando os arquivos csv baixados do site da Câmara dos Deputados

In [29]:
arquivos = os.listdir('ArquivosCSV-2019')
arquivos

['deputados.csv',
 'eventos-2019.csv',
 'eventosOrgaos-2019.csv',
 'eventosPresencaDeputados-2019.csv',
 'eventosRequerimentos-2019.csv',
 'frentes.csv',
 'frentesDeputados.csv',
 'legislaturas.csv',
 'legislaturasMesas.csv',
 'orgaos.csv',
 'proposicoes-2019.csv',
 'proposicoesAutores-2019.csv',
 'proposicoesTemas-2019.csv',
 'README',
 'votacoes-2019.csv',
 'votacoesObjetos-2019.csv',
 'votacoesOrientacoes-2019.csv',
 'votacoesProposicoes-2019.csv',
 'votacoesVotos-2019.csv']

### Limpeza no Arquivo: deputados.csv

In [20]:
file_deputados = 'ArquivosCSV-2019/' + arquivos[0]
db_deputados = pd.read_csv(file_deputados, delimiter=';')

Removendo colunas que não serão utilizadas:
* cpf : Essa coluna estava vazia
* urlRedeSocial, urlWebsite : Não possui informações de todos candidatos além de que não será utilizada no momento
* dataNascimento, dataFalecimento: Informações irrelevantes no momento
* idLegislaturaInicial : Informação irrelevantes no momento

In [21]:
to_drop = [
    'cpf', 
    'urlRedeSocial', 
    'urlWebsite', 
    'dataNascimento',
    #'nome',
    #'nomeCivil',
    #'municipioNascimento'
    'dataFalecimento', 
    'idLegislaturaInicial',
]
          
db_deputados = db_deputados.drop(columns=to_drop)
db_deputados.columns

Index(['uri', 'nome', 'idLegislaturaFinal', 'nomeCivil', 'siglaSexo',
       'ufNascimento', 'municipioNascimento'],
      dtype='object')

Queremos somente os Deputados atuais, ou seja, o que estão fazendo parte da câmara na 56º Legislatura

In [22]:
db_deputados = db_deputados[db_deputados.idLegislaturaFinal == 56]
db_deputados

,uri,nome,idLegislaturaFinal,nomeCivil,siglaSexo,ufNascimento,municipioNascimento
2824,https://dadosabertos.camara.leg.br/api/v2/depu...,Danilo Forte,56,FRANCISCO DANILO BASTOS FORTE,M,CE,Fortaleza
2842,https://dadosabertos.camara.leg.br/api/v2/depu...,Norma Ayub,56,NORMA AYUB ALVES,F,ES,Vitória
2845,https://dadosabertos.camara.leg.br/api/v2/depu...,Fausto Pinato,56,FAUSTO RUY PINATO,M,SP,Fernandópolis
2847,https://dadosabertos.camara.leg.br/api/v2/depu...,Iracema Portella,56,IRACEMA MARIA PORTELLA NUNES NOGUEIRA LIMA,F,PI,Teresina
2852,https://dadosabertos.camara.leg.br/api/v2/depu...,Fábio Henrique,56,FABIO HENRIQUE SANTANA DE CARVALHO,M,SE,Simão Dias
...,...,...,...,...,...,...,...
7622,https://dadosabertos.camara.leg.br/api/v2/depu...,Dr. Agripino Magalhães,56,AGRIPINO RODRIGUES GOMES MAGALHÃES,M,CE,Boa Viagem
7623,https://dadosabertos.camara.leg.br/api/v2/depu...,Deuzinho Filho,56,FRANCISCO DEUZINHO DE OLIVEIRA FILHO,M,CE,Caucaia
7624,https://dadosabertos.camara.leg.br/api/v2/depu...,Nilson F. Stainsack,56,NILSON FRANCISCO STAINSACK,M,SC,Presidente Getúlio
7625,https://dadosabertos.camara.leg.br/api/v2/depu...,Renato Queiroz,56,RENATO ANDRADE QUEIROZ,M,RR,Boa Vista


Uma informação importante que será necessári é o id do Deputado. Essa informação não possui uma coluna para identifica-la, porém a coluna uri, que referencia os endpoints com informações de cada deputado, realiza a pesquisa das informações utilizando o id do candidato. Poderíamos realizar um simples parse para obter o id, entretando preferimos realizar a coleta a partir do endpoint pois poderia trazer mais informações do deputado. Atualmente a única informação que está sendo lida é o id do deputado mas a estrutura utilizada permite pegar as demais informações com facilidade.

Iremos coletar o id de cada deputado e adicionar uma coluna "deputado_id" no dataframe

In [ ]:
lista_ids = []
for uri in db_deputados.uri:
    try:
        json = requests.get(uri).json()
        deputado_id = json['dados']['id']
    except:
        deputado_id = None
        
    lista_ids.append(deputado_id)
    
db_deputados.loc[:,'deputado_id'] = lista_ids

Em seguida, podemos droppar a coluna uri

In [ ]:
db_deputados = db_deputados.drop(columns=['uri'])

In [25]:
db_deputados = db_deputados.applymap(limpa_url)
db_deputados.rename(columns={ 'uri' : 'deputado_id' }, inplace=True)

Por fim, vamos criar um novo documento com as informações 

In [26]:
db_deputados.to_csv('Arquivos Limpos/' + 'deputados.csv', encoding='utf-8')

### Limpeza no Arquivo: votacoes-2019.csv

In [30]:
file_votacoes = 'ArquivosCSV-2019/votacoes-2019.csv'
votacoes = pd.read_csv(file_votacoes, delimiter=';')

Removendo colunas que não serão utilizadas:

In [31]:
to_drop = ['uri', 
           'data', 
           'dataHoraRegistro', 
           'uriOrgao',
           'siglaOrgao',
           'uriEvento',
           'descricao',
           'ultimaAberturaVotacao_dataHoraRegistro',
           'ultimaAberturaVotacao_descricao',
           'ultimaApresentacaoProposicao_dataHoraRegistro',
           'ultimaApresentacaoProposicao_descricao',
           'ultimaApresentacaoProposicao_uriProposicao'
          ]

votacoes.drop(to_drop, inplace=True, axis=1)

Descartando linhas com valores faltantes

In [32]:
votacoes.dropna(inplace=True,axis=0)

Dataframe limpo:

In [38]:
votacoes.head()

,id,idOrgao,idEvento,aprovacao,votosSim,votosNao,votosOutros,ultimaApresentacaoProposicao_idProposicao
0,2179189-45,180,54502,0.0,21,238,32,2186805
1,2179189-48,180,54502,0.0,0,0,0,2186805
2,2179189-59,180,54502,0.0,53,294,47,2191685
3,2179189-62,180,54502,1.0,0,0,0,2191685
4,2179189-73,180,54502,0.0,0,0,0,2191771


In [57]:
votacoes.to_csv('Arquivos Limpos/' + 'votacoes.csv', encoding='utf-8')

### Limpeza no Arquivo: votacoesVotos-2019.csv

In [41]:
votacoesVotos = pd.read_csv('ArquivosCSV-2019/votacoesVotos-2019.csv', delimiter=';') 

Removendo colunas que não serão utilizadas:

In [42]:
to_drop = ['uriVotacao', 
           'dataHoraVoto', 
           'deputado_uri', 
           'deputado_nome',
           'deputado_idLegislatura',
           'deputado_urlFoto'
          ]

votacoesVotos.drop(to_drop, inplace=True, axis=1)

Função que limpa os urls

In [43]:
def limpa_url(item):
    if isinstance(item, str):
        if 'https' in item:
            return item.split('/')[-1]
        elif item == "Não":
            return -1
        elif item == "Sim":
            return 1
        elif item == "Obstrução":
            return 0
        elif item == "Abstenção":
            return 0
        elif item == "Artigo 17":
            return 0
        else:
            return unidecode(item)
    else:
        return item

Limpando e renomeando a coluna cos urls dos partidos

In [44]:
votacoesVotos = votacoesVotos.applymap(limpa_url)
votacoesVotos.rename(columns={ 'deputado_uriPartido' : 'pardido_id' }, inplace=True)

Dataframe limpo:

In [47]:
votacoesVotos['voto'].sum()

-1637

In [81]:
votacoesVotos.to_csv('Arquivos Limpos/' + 'votacoesVotos.csv', encoding='utf-8')

In [51]:
proposicoesAutores = pd.read_csv('ArquivosCSV-2019/proposicoesAutores-2019.csv', delimiter=';') 

In [52]:
to_drop = ['uriProposicao', 
           'uriAutor', 
           'codTipoAutor',
           'tipoAutor',
           'nomeAutor',
           'siglaPartidoAutor',
           'uriPartidoAutor',
           'siglaUFAutor',
           'ordemAssinatura',
           'proponente'
          ]

proposicoesAutores.drop(to_drop, inplace=True, axis=1)

In [55]:
proposicoesAutores.head()

,idProposicao,idDeputadoAutor
48,2237302,141450.0
49,2237301,141450.0
50,2237300,141450.0
51,2237299,204521.0
52,2237298,204375.0


In [54]:
proposicoesAutores.dropna(inplace=True,axis=0)

In [60]:
proposicoesAutores.to_csv('Arquivos Limpos/' + 'proposicoesAutores.csv', encoding='utf-8')